# Calibrate all snapshots (BTC and ETH) → Excel workbook

This notebook runs the **batch calibration pipeline** that calibrates **Black**, **Heston**, and **SVCJ** to all
`deribit_options_snapshot_*.csv` files in `data/` and persists results to a single Excel workbook.

Outputs in `calibration_results.xlsx`:

- `black_params`, `heston_params`, `svcj_params`: one row per (snapshot timestamp, currency)
- `train_data`, `test_data`: the option rows used, each with model price columns

Key features:

- **Resume-safe**: if the workbook exists, each currency resumes from the first snapshot after the latest timestamp
  already present in `black_params`.
- **Crash-safe writes**: workbook flush is atomic (temp file → replace).
- **Warm-start**: each worker chunk warm-starts from the most recent successful params prior to the chunk start.
- **Threading**: worker threads may finish out-of-order, but commits are written in chronological order.


In [ ]:
# Imports and project path setup
from __future__ import annotations

import os
import sys
from pathlib import Path

import pandas as pd

# Recommended when using multiple workers: avoid BLAS/OMP oversubscription.
# NOTE: for best effect, set these before importing numpy/scipy in a fresh kernel.
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# --- locate project root (folder that contains `src/` and `data/`)
ROOT = Path.cwd().resolve()
candidates = [ROOT, ROOT.parent, *ROOT.parents]
for c in candidates:
    if (c / "src").exists() and (c / "data").exists():
        ROOT = c
        break

assert (ROOT / "src").exists() and (ROOT / "data").exists(), (
    f"Could not find project root from cwd={Path.cwd()}"
)

# Make project importable
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.batch_runner import BatchConfig, run_all_snapshots_to_excel
from src.calibration import WeightConfig
from src.inverse_fft_pricer import FFTParams

pd.set_option("display.max_columns", 200)
print("Project root:", ROOT)


In [ ]:
# Global configuration

OUTPUT_XLSX = ROOT / "calibration_results.xlsx"

cfg = BatchConfig(
    project_root=ROOT,
    output_xlsx=OUTPUT_XLSX,
    resume=True,
    save_every_n_files=10,
    smoke_test_max_files_per_currency=None,  # set to 1–2 for quick smoke tests

    # Filtering
    filter_rules=dict(
        require_bid_ask=True,
        min_time_to_maturity=1/365,
        max_time_to_maturity=None,
        min_open_interest=1.0,
        min_vega=0.0,
        max_rel_spread=0.50,
        moneyness_range=(0.5, 2.0),
        drop_synthetic_underlyings=False,
    ),
    min_options_after_filter=50,

    # Weights: r_i = w_i * (P_model - P_mkt)
    weight_config=WeightConfig(
        use_spread=True,
        use_vega=False,
        use_open_interest=False,
        spread_power=1.0,
        vega_power=0.5,
        oi_power=0.5,
        eps_spread=1e-6,
        eps_other=1e-12,
        cap=1e6,
    ),

    # FFT base (per-expiry b is computed per snapshot)
    fft_base=FFTParams(
        N=2**12,
        eta=0.10,
        alpha=1.5,
        b=-10.0,
        use_simpson=True,
    ),

    # Calibration knobs
    train_frac=0.70,
    global_random_seed=123,
    max_nfev=dict(black=200, heston=200, svcj=200),

    # Runtime throttles (optional)
    runtime_top_expiries_by_oi=None,
    runtime_max_options=None,

    # Parallelism
    n_workers=8,
    limit_internal_threads=True,
    internal_num_threads=1,

    currencies=("BTC", "ETH"),
    verbose=True,
)

In [ ]:
# Run calibrations and persist to Excel
wb = run_all_snapshots_to_excel(cfg)
print("Done. Workbook at:", cfg.output_xlsx)
